# Stage 2 위조 비율 실험: MIL Bag Generation (5/10/20/30/50% Forgery)

이 노트북은 Stage 1에서 추출한 ArcFace 임베딩을 이용해 Multiple Instance Learning(MIL) 학습을 위한 **다양한 위조 비율** Bag 데이터를 생성합니다.

**핵심 개선:**
- **다양한 위조 비율**: 5%, 10%, 20%, 30%, 50%로 양성 bag 내 B 작성자 비율 조절
- **두 가지 실험 모드**: Matched (각 비율별 독립 학습) vs Shift (30% 학습, 다양한 비율 평가)
- **고정 윈도우**: (win=5, stride=1) → 10개 인스턴스 → 각 bag은 (10, 5, 256)
- **50/50 균형**: 각 split에서 정확히 50% Positive, 50% Negative 비율

In [1]:
# 라이브러리 임포트 및 환경 설정
import os, random, pickle, numpy as np, pandas as pd
from collections import defaultdict

# GPU 설정 (선택적)
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = os.getenv('MIL_STAGE2_GPU', '3')

# 재현성을 위한 시드 설정
SEED_BASE = 42
np.random.seed(SEED_BASE)
random.seed(SEED_BASE)

print(f'환경 설정 완료: GPU={os.environ.get("CUDA_VISIBLE_DEVICES", "N/A")}, SEED={SEED_BASE}')

환경 설정 완료: GPU=3, SEED=42


In [2]:
# 경로 설정 및 데이터 로딩
embedding_dir = '/workspace/MIL/data/processed/embeddings'
raw_meta_csv  = '/workspace/MIL/data/raw/naver_ocr.csv'  # 선택 사항
bags_dir      = '/workspace/MIL/data/processed/bags'
os.makedirs(bags_dir, exist_ok=True)

margin_value = '0.4'
rng_global = np.random.default_rng(SEED_BASE)

# CSV 로딩 함수
def load_split_csv(split):
    csv_path = os.path.join(embedding_dir, f'mil_arcface_margin_{margin_value}_{split}_data.csv')
    df = pd.read_csv(csv_path)
    label_col = 'label' if 'label' in df.columns else 'author_id'
    emb_cols  = [c for c in df.columns if c.startswith('embedding')]
    assert len(emb_cols) > 0, "No embedding_* columns found."
    return df, label_col, emb_cols

# 데이터 로딩
train_df, label_col, emb_cols = load_split_csv('train')
val_df,   _,         _        = load_split_csv('val')
test_df,  _,         _        = load_split_csv('test')
embed_dim = len(emb_cols)

print(f"데이터 로딩 완료:")
print(f"  - Embedding 차원: {embed_dim}")
print(f"  - Label 컬럼: {label_col}")
print(f"  - Train: {len(train_df)}, Val: {len(val_df)}, Test: {len(test_df)}")

# 선택적 원본 메타데이터 로딩
try:
    original_df = pd.read_csv(raw_meta_csv)
    print(f"  - 원본 메타데이터: {len(original_df)} rows")
    has_raw_meta = True
except Exception:
    print(f"  - 원본 메타데이터: 없음")
    has_raw_meta = False

데이터 로딩 완료:
  - Embedding 차원: 256
  - Label 컬럼: label
  - Train: 208233, Val: 70533, Test: 72457
  - 원본 메타데이터: 556128 rows


In [3]:
# ==============================================================
# Stage 2 위조 비율 실험: MIL Bags (5/10/20/30% Forgery)
# ==============================================================

# 작성자별 인덱스 구축
def build_writer_index(df, label_col, emb_cols):
    w2 = {}
    for wid, g in df.groupby(label_col):
        w2[int(wid)] = {
            'emb': g[emb_cols].to_numpy(dtype=np.float32),
            'paths': g['path'].tolist() if 'path' in g.columns else [''] * len(g),
            'idx': g.index.to_list()
        }
    return w2

train_writers = build_writer_index(train_df, label_col, emb_cols)
val_writers   = build_writer_index(val_df,   label_col, emb_cols)
test_writers  = build_writer_index(test_df,  label_col, emb_cols)

def list_writer_ids(wdict): 
    return list(wdict.keys())

print(f"작성자 인덱스 구축 완료:")
print(f"  - Train writers: {len(train_writers)}")
print(f"  - Val writers: {len(val_writers)}")
print(f"  - Test writers: {len(test_writers)}")

# 샘플링 헬퍼 함수들
def sample_k(n, k, rng, replace_if_needed=True):
    """n개 중에서 k개 샘플링 (부족하면 중복 허용)"""
    if (not replace_if_needed) and n >= k:
        return rng.choice(n, size=k, replace=False).tolist()
    # 부족하면 중복 허용
    return rng.choice(n, size=k, replace=True).tolist()

def sliding_windows(seq, win=5, stride=1):
    """슬라이딩 윈도우로 시퀀스 분할"""
    # seq: list of tuples (emb, wid, path, orig_idx)
    windows, metas = [], []
    for i in range(0, len(seq) - win + 1, stride):
        chunk = seq[i:i+win]
        windows.append(np.stack([e for (e,_,_,_) in chunk], axis=0))  # (5, D)
        metas.append({
            'window_idx': i,
            'word_indices': [oi for (_,_,_,oi) in chunk],
            'word_paths':   [p  for (_,_,p, _) in chunk],
            'writer_ids':   [w  for (_,w,_, _) in chunk],
        })
    return windows, metas

def pack(words, wid, W):
    """단어 인덱스를 임베딩 튜플로 변환"""
    emb, paths, idxs = W['emb'], W['paths'], W['idx']
    return [(emb[w], wid, paths[w], idxs[w]) for w in words]

print("✓ 헬퍼 함수 정의 완료")

작성자 인덱스 구축 완료:
  - Train writers: 180
  - Val writers: 60
  - Test writers: 60
✓ 헬퍼 함수 정의 완료


In [4]:
# ==== [NEW] Forgery ratio utilities ====
TOTAL_TOKENS_POS = 14        # 양성 문서 총 단어 수 (기존 7+7)
TOK_NEG = 14                 # 음성 문서 총 단어 수 (기존 유지)
WIN = 5
STRIDE = 1
INSTANCES_PER_BAG = 10

def ratio_to_counts(pos_ratio: float, total_tokens: int = TOTAL_TOKENS_POS):
    """
    pos_ratio(0~0.5 가 권장)를 토대로 B작성자 토큰 수를 계산.
    최소 1개씩은 A/B가 들어가도록 안전장치.
    예: 0.05 → B=1, A=13 / 0.30 → B=4, A=10
    """
    b = int(round(total_tokens * pos_ratio))
    b = max(1, min(b, total_tokens - 1))  # A/B 최소 1개 보장
    a = total_tokens - b
    return a, b

def ratio_tag(pos_ratio: float) -> str:
    """파일명 태그: 0.05 → '05p', 0.30 → '30p'"""
    return f"{int(round(pos_ratio*100)):02d}p"

print("✓ 위조 비율 유틸리티 함수 정의 완료")
print(f"  - 총 토큰: {TOTAL_TOKENS_POS}개")
print(f"  - 윈도우: (win={WIN}, stride={STRIDE})")
print(f"  - 인스턴스/Bag: {INSTANCES_PER_BAG}개")

✓ 위조 비율 유틸리티 함수 정의 완료
  - 총 토큰: 14개
  - 윈도우: (win=5, stride=1)
  - 인스턴스/Bag: 10개


In [5]:
# Bag 생성 함수 (위조 비율 지원)

def make_negative_bag(wid, W, rng):
    """단일 작성자 Bag (레이블 0)"""
    emb, paths, idxs = W['emb'], W['paths'], W['idx']
    sel = sample_k(len(emb), TOK_NEG, rng, replace_if_needed=True)
    seq = pack(sel, wid, W)
    wins, metas = sliding_windows(seq, WIN, STRIDE)
    bag = np.stack(wins[:INSTANCES_PER_BAG], axis=0)  # (10, 5, D)
    return bag, metas[:INSTANCES_PER_BAG], [int(wid)]

def make_positive_bag(widA, widB, WA, WB, rng, pos_ratio: float):
    """
    복수 작성자 Bag (레이블 1): A/B 토큰 수를 pos_ratio로 결정 → 전체 셔플 → 윈도우
    기존과 동일하게 인스턴스 안에 두 작성자 토큰이 섞일 수 있음(stage2_baseline 규칙 유지).
    """
    # A/B 토큰 수 결정
    TOK_POS_A, TOK_POS_B = ratio_to_counts(pos_ratio, total_tokens=TOTAL_TOKENS_POS)

    embA, pathsA, idxA = WA['emb'], WA['paths'], WA['idx']
    embB, pathsB, idxB = WB['emb'], WB['paths'], WB['idx']
    
    # 가능한 경우 중복없이 뽑아(현실성↑), 부족할 때만 중복 허용
    selA = sample_k(len(embA), TOK_POS_A, rng, replace_if_needed=False)
    selB = sample_k(len(embB), TOK_POS_B, rng, replace_if_needed=False)

    seqA = [(embA[i], int(widA), pathsA[i], idxA[i]) for i in selA]
    seqB = [(embB[i], int(widB), pathsB[i], idxB[i]) for i in selB]
    
    seq = seqA + seqB
    rng.shuffle(seq)

    # 윈도우 생성
    wins, metas = sliding_windows(seq, WIN, STRIDE)
    bag = np.stack(wins[:INSTANCES_PER_BAG], axis=0)

    # 메타에 '실제' 토큰 비율 기록(검증용)
    token_writer_ids = [w for (_,w,_,_) in seq]
    a_count = token_writer_ids.count(int(widA))
    b_count = token_writer_ids.count(int(widB))
    realized_ratio = b_count / (a_count + b_count)

    return bag, metas[:INSTANCES_PER_BAG], [int(widA), int(widB)], {
        'target_ratio': float(pos_ratio),
        'tokens_A': int(a_count),
        'tokens_B': int(b_count),
        'realized_ratio': float(realized_ratio)
    }

print("✓ Bag 생성 함수 정의 완료 (위조 비율 지원)")
print(f"  - Negative: {TOK_NEG}개 단어 → {INSTANCES_PER_BAG}개 인스턴스")
print(f"  - Positive: 비율별 A+B개 단어 셔플 → {INSTANCES_PER_BAG}개 인스턴스")

✓ Bag 생성 함수 정의 완료 (위조 비율 지원)
  - Negative: 14개 단어 → 10개 인스턴스
  - Positive: 비율별 A+B개 단어 셔플 → 10개 인스턴스


In [6]:
# Split 생성 함수 (위조 비율 지원)
def generate_split(name, WDICT, neg_per_writer=10, pos_per_writer=10, seed=42, pos_ratio=0.30):
    """
    간단한 베이스라인 split 생성. pos_ratio만 바꿔 다양한 위조 비율 세트 생성.
    """
    rng = np.random.default_rng(seed)
    writer_ids = list_writer_ids(WDICT)
    bags, labels, metadata = [], [], []
    
    print(f"  {name} split 생성 중... (Writers: {len(writer_ids)}, pos_ratio={pos_ratio:.2f})")
    
    # Negative bags (단일 작성자)
    for wid in writer_ids:
        for _ in range(neg_per_writer):
            bag, metas, authors = make_negative_bag(wid, WDICT[wid], rng)
            bags.append(bag)
            labels.append(0)
            metadata.append({
                'authors': authors, 
                'bag_type': 'negative',
                'instances': metas,
                'target_ratio': 0.0,
                'realized_ratio': 0.0,
                'tokens_A': TOK_NEG,
                'tokens_B': 0
            })
    
    # Positive bags (복수 작성자, 각 A당 pos_per_writer개, 파트너는 랜덤)
    for widA in writer_ids:
        for _ in range(pos_per_writer):
            widB = rng.choice([w for w in writer_ids if w != widA])
            bag, metas, authors, ratio_meta = make_positive_bag(
                widA, widB, WDICT[widA], WDICT[widB], rng, pos_ratio=pos_ratio
            )
            bags.append(bag)
            labels.append(1)
            md = {
                'authors': authors, 
                'bag_type': 'positive',
                'instances': metas
            }
            md.update(ratio_meta)  # target_ratio / realized_ratio / tokens_A / tokens_B
            metadata.append(md)
    
    # 전체 셔플
    idx = rng.permutation(len(labels))
    bags = [bags[i] for i in idx]
    labels = [int(labels[i]) for i in idx]
    metadata = [metadata[i] for i in idx]
    
    # 요약 출력
    n_pos = sum(labels)
    n_tot = len(labels)
    print(f"    → Total: {n_tot}, Positive: {n_pos} ({n_pos/n_tot*100:.1f}%), Negative: {n_tot-n_pos}")
    
    # 간단 검증
    assert len(bags) == len(labels) == len(metadata), "Length mismatch"
    assert bags[0].shape == (INSTANCES_PER_BAG, WIN, embed_dim), f"Shape mismatch: {bags[0].shape}"
    
    return bags, labels, metadata

print("✓ Split 생성 함수 정의 완료 (위조 비율 지원)")

✓ Split 생성 함수 정의 완료 (위조 비율 지원)


In [7]:
# 저장 함수 (비율 태그 포함)
def save_split(bags, labels, meta, tag, pos_ratio, compat_copy=True):
    rtag = ratio_tag(pos_ratio)  # 예: '05p', '30p'
    base = os.path.join(bags_dir, f"bags_arcface_margin_{margin_value}_{rtag}_baseline_{tag}.pkl")
    with open(base, 'wb') as f:
        pickle.dump({'bags': bags, 'labels': labels, 'metadata': meta}, f)
    print(f"💾 저장: {os.path.basename(base)}")
    
    if compat_copy:
        # Stage3 호환용 별칭도 비율 태그를 포함시켜 충돌 방지
        alias = os.path.join(bags_dir, f"bags_arcface_margin_{margin_value}_{rtag}_random_{tag}.pkl")
        with open(alias, 'wb') as f:
            pickle.dump({'bags': bags, 'labels': labels, 'metadata': meta}, f)
        print(f"↪️  호환 복사: {os.path.basename(alias)}")

print("✓ 저장 함수 정의 완료 (비율 태그 지원)")

✓ 저장 함수 정의 완료 (비율 태그 지원)


In [8]:
# ==== [RUN] Generate datasets for multiple ratios ====
NEG_PW = 10
POS_PW = 10

RATIOS = [0.05, 0.10, 0.20, 0.30, 0.50]

print("🔄 Matched 모드: Train/Val/Test 모두 동일 비율")
for i, r in enumerate(RATIOS):
    print(f"\n=== [Matched] ratio={r:.2f} ({ratio_tag(r)}) ===")
    train_bags, train_labels, train_meta = generate_split('Train', train_writers, NEG_PW, POS_PW, seed=SEED_BASE+100*i+0, pos_ratio=r)
    val_bags,   val_labels,   val_meta   = generate_split('Val',   val_writers,   NEG_PW, POS_PW, seed=SEED_BASE+100*i+10, pos_ratio=r)
    test_bags,  test_labels,  test_meta  = generate_split('Test',  test_writers,  NEG_PW, POS_PW, seed=SEED_BASE+100*i+20, pos_ratio=r)

    save_split(train_bags, train_labels, train_meta, 'train', pos_ratio=r, compat_copy=True)
    save_split(val_bags,   val_labels,   val_meta,   'val',   pos_ratio=r, compat_copy=True)
    save_split(test_bags,  test_labels,  test_meta,  'test',  pos_ratio=r, compat_copy=True)

print("\n🔄 Shift 모드: Train=30% 고정, Val/Test만 비율 변경")
r_train = 0.30
print(f"\n=== [Shift] Train ratio={r_train:.2f} ({ratio_tag(r_train)}) ===")
train_bags_30, train_labels_30, train_meta_30 = generate_split('Train', train_writers, NEG_PW, POS_PW, seed=SEED_BASE+999, pos_ratio=r_train)
val_bags_30,   val_labels_30,   val_meta_30   = generate_split('Val',   val_writers,   NEG_PW, POS_PW, seed=SEED_BASE+1009, pos_ratio=r_train)
# 학습용 세트 저장
save_split(train_bags_30, train_labels_30, train_meta_30, 'train_shiftbase', pos_ratio=r_train, compat_copy=True)
save_split(val_bags_30,   val_labels_30,   val_meta_30,   'val_shiftbase',   pos_ratio=r_train, compat_copy=True)

# 평가용(다양한 비율) Val/Test만 추가 생성
for i, r in enumerate(RATIOS):
    print(f"\n--- [Shift] Eval-only ratio={r:.2f} ({ratio_tag(r)}) ---")
    val_bags_s,  val_labels_s,  val_meta_s  = generate_split('Val',  val_writers,  NEG_PW, POS_PW, seed=SEED_BASE+2000+10*i, pos_ratio=r)
    test_bags_s, test_labels_s, test_meta_s = generate_split('Test', test_writers, NEG_PW, POS_PW, seed=SEED_BASE+2000+20*i, pos_ratio=r)
    save_split(val_bags_s,  val_labels_s,  val_meta_s,  f'val_shift_{ratio_tag(r)}',  pos_ratio=r, compat_copy=True)
    save_split(test_bags_s, test_labels_s, test_meta_s, f'test_shift_{ratio_tag(r)}', pos_ratio=r, compat_copy=True)

🔄 Matched 모드: Train/Val/Test 모두 동일 비율

=== [Matched] ratio=0.05 (05p) ===
  Train split 생성 중... (Writers: 180, pos_ratio=0.05)
    → Total: 3600, Positive: 1800 (50.0%), Negative: 1800
  Val split 생성 중... (Writers: 60, pos_ratio=0.05)
    → Total: 1200, Positive: 600 (50.0%), Negative: 600
  Test split 생성 중... (Writers: 60, pos_ratio=0.05)
    → Total: 1200, Positive: 600 (50.0%), Negative: 600
💾 저장: bags_arcface_margin_0.4_05p_baseline_train.pkl
↪️  호환 복사: bags_arcface_margin_0.4_05p_random_train.pkl
💾 저장: bags_arcface_margin_0.4_05p_baseline_val.pkl
↪️  호환 복사: bags_arcface_margin_0.4_05p_random_val.pkl
💾 저장: bags_arcface_margin_0.4_05p_baseline_test.pkl
↪️  호환 복사: bags_arcface_margin_0.4_05p_random_test.pkl

=== [Matched] ratio=0.10 (10p) ===
  Train split 생성 중... (Writers: 180, pos_ratio=0.10)
    → Total: 3600, Positive: 1800 (50.0%), Negative: 1800
  Val split 생성 중... (Writers: 60, pos_ratio=0.10)
    → Total: 1200, Positive: 600 (50.0%), Negative: 600
  Test split 생성 중... (Writer

In [9]:
# 생성된 데이터 검증 및 샘플 확인
print("🔍 생성된 데이터 검증:")

# 각 비율별 샘플 메타데이터 확인
sample_ratios = [0.05, 0.50]  # 극단 비율들만 확인

for r in sample_ratios:
    print(f"\n📋 {ratio_tag(r)} 비율 샘플 메타데이터:")
    
    # 해당 비율로 생성된 작은 샘플 생성 (검증용)
    test_bags_sample, test_labels_sample, test_meta_sample = generate_split(
        'Test_Sample', test_writers, 1, 1, seed=SEED_BASE+9999, pos_ratio=r
    )
    
    # Negative bag 샘플
    neg_sample = next(meta for meta, label in zip(test_meta_sample, test_labels_sample) if label == 0)
    print(f"  Negative bag:")
    print(f"    - Authors: {neg_sample['authors']} (개수: {len(neg_sample['authors'])})")
    print(f"    - Type: {neg_sample['bag_type']}")
    print(f"    - Target ratio: {neg_sample['target_ratio']:.2f}")
    print(f"    - Tokens A/B: {neg_sample['tokens_A']}/{neg_sample['tokens_B']}")
    
    # Positive bag 샘플
    pos_sample = next(meta for meta, label in zip(test_meta_sample, test_labels_sample) if label == 1)
    print(f"  Positive bag:")
    print(f"    - Authors: {pos_sample['authors']} (개수: {len(pos_sample['authors'])})")
    print(f"    - Type: {pos_sample['bag_type']}")
    print(f"    - Target ratio: {pos_sample['target_ratio']:.2f}")
    print(f"    - Realized ratio: {pos_sample['realized_ratio']:.2f}")
    print(f"    - Tokens A/B: {pos_sample['tokens_A']}/{pos_sample['tokens_B']}")
    print(f"    - 첫 인스턴스 작성자들: {pos_sample['instances'][0]['writer_ids']}")

print(f"\n✅ 모든 검증 통과 - 위조 비율별 데이터 준비 완료!")
print(f"🚀 Stage 3에서 다음과 같이 활용:")
print(f"  - Matched 모드: 각 비율별 독립 학습 {len(RATIOS)}회")
print(f"  - Shift 모드: 30% 학습 → 5/10/20/30/50% 평가")

🔍 생성된 데이터 검증:

📋 05p 비율 샘플 메타데이터:
  Test_Sample split 생성 중... (Writers: 60, pos_ratio=0.05)
    → Total: 120, Positive: 60 (50.0%), Negative: 60
  Negative bag:
    - Authors: [260] (개수: 1)
    - Type: negative
    - Target ratio: 0.00
    - Tokens A/B: 14/0
  Positive bag:
    - Authors: [271, 290] (개수: 2)
    - Type: positive
    - Target ratio: 0.05
    - Realized ratio: 0.07
    - Tokens A/B: 13/1
    - 첫 인스턴스 작성자들: [271, 290, 271, 271, 271]

📋 50p 비율 샘플 메타데이터:
  Test_Sample split 생성 중... (Writers: 60, pos_ratio=0.50)
    → Total: 120, Positive: 60 (50.0%), Negative: 60
  Negative bag:
    - Authors: [260] (개수: 1)
    - Type: negative
    - Target ratio: 0.00
    - Tokens A/B: 14/0
  Positive bag:
    - Authors: [271, 290] (개수: 2)
    - Type: positive
    - Target ratio: 0.50
    - Realized ratio: 0.50
    - Tokens A/B: 7/7
    - 첫 인스턴스 작성자들: [271, 290, 271, 271, 271]

✅ 모든 검증 통과 - 위조 비율별 데이터 준비 완료!
🚀 Stage 3에서 다음과 같이 활용:
  - Matched 모드: 각 비율별 독립 학습 5회
  - Shift 모드: 30% 학습 → 5/10/2

In [10]:
# 최종 요약
print("=" * 80)
print("🎯 Stage 2 위조 비율 실험 완료!")
print("=" * 80)
print("✅ 생성된 데이터셋:")

print("\n📊 Matched 모드 (각 비율별 독립):")
for r in RATIOS:
    rtag = ratio_tag(r)
    expected_a, expected_b = ratio_to_counts(r)
    print(f"  • {rtag} ({r:.0%}): A={expected_a}, B={expected_b} 토큰")
    print(f"    - bags_arcface_margin_{margin_value}_{rtag}_baseline_*.pkl")
    print(f"    - bags_arcface_margin_{margin_value}_{rtag}_random_*.pkl (호환)")

print("\n📊 Shift 모드 (30% 학습, 다양한 평가):")
print(f"  • 학습용: 30p (30%) - A=10, B=4 토큰")
print(f"    - bags_arcface_margin_{margin_value}_30p_baseline_train_shiftbase.pkl")
print(f"    - bags_arcface_margin_{margin_value}_30p_random_train_shiftbase.pkl")
print(f"    - bags_arcface_margin_{margin_value}_30p_baseline_val_shiftbase.pkl")
print(f"    - bags_arcface_margin_{margin_value}_30p_random_val_shiftbase.pkl")
print(f"  • 평가용:")
for r in RATIOS:
    rtag = ratio_tag(r)
    expected_a, expected_b = ratio_to_counts(r)
    print(f"    - {rtag}_shift ({r:.0%}): A={expected_a}, B={expected_b}")

print("\n🎯 연구 목적:")
print("  • 낮은 위조 비율(5%, 10%)에서 모델 성능 평가")
print("  • Recall 유지 능력 측정 (위조 놓치지 않기)")
print("  • 도메인 적응 능력 평가 (Shift 모드)")

print("\n🚀 다음 단계: Stage 3에서 AB-MIL 학습 및 비교 분석")

🎯 Stage 2 위조 비율 실험 완료!
✅ 생성된 데이터셋:

📊 Matched 모드 (각 비율별 독립):
  • 05p (5%): A=13, B=1 토큰
    - bags_arcface_margin_0.4_05p_baseline_*.pkl
    - bags_arcface_margin_0.4_05p_random_*.pkl (호환)
  • 10p (10%): A=13, B=1 토큰
    - bags_arcface_margin_0.4_10p_baseline_*.pkl
    - bags_arcface_margin_0.4_10p_random_*.pkl (호환)
  • 20p (20%): A=11, B=3 토큰
    - bags_arcface_margin_0.4_20p_baseline_*.pkl
    - bags_arcface_margin_0.4_20p_random_*.pkl (호환)
  • 30p (30%): A=10, B=4 토큰
    - bags_arcface_margin_0.4_30p_baseline_*.pkl
    - bags_arcface_margin_0.4_30p_random_*.pkl (호환)
  • 50p (50%): A=7, B=7 토큰
    - bags_arcface_margin_0.4_50p_baseline_*.pkl
    - bags_arcface_margin_0.4_50p_random_*.pkl (호환)

📊 Shift 모드 (30% 학습, 다양한 평가):
  • 학습용: 30p (30%) - A=10, B=4 토큰
    - bags_arcface_margin_0.4_30p_baseline_train_shiftbase.pkl
    - bags_arcface_margin_0.4_30p_random_train_shiftbase.pkl
    - bags_arcface_margin_0.4_30p_baseline_val_shiftbase.pkl
    - bags_arcface_margin_0.4_30p_random_val_sh